In [5]:
import os
import sys

sys.path.append("../..")

import numpy as np 
import pandas as pd
from mikasa.io import load_pickle, dump_pickle

In [6]:
oof = pd.read_csv('../data/working/oof.csv')
y = load_pickle("../data/feature/target.pkl")

Load pickle from ../data/feature/target.pkl


In [7]:
oof

,LR,RF,LGBM,XGB
0,0.092186,0.126112,0.066892,0.082665
1,0.914150,0.914171,0.979135,0.988178
2,0.617613,0.424315,0.514993,0.598507
3,0.891066,0.856954,0.969985,0.990880
4,0.081696,0.131179,0.178577,0.188030
...,...,...,...,...
886,0.289567,0.207854,0.168765,0.164623
887,0.951497,0.805728,0.910546,0.951710
888,0.499508,0.394614,0.522341,0.281549
889,0.584714,0.431946,0.424761,0.541064


In [48]:
print("Simple Average Score:")
accuracy_score(y, (oof.mean(axis=1) > 0.5))

Simple Average Score:


0.8361391694725028

## Optimize Weight

In [77]:
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

predictions = oof.T.to_numpy()
len_prediction = len(predictions)

def metric_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, predictions):
        final_prediction += weight * prediction
        
    final_prediction = accuracy_score(y, (final_prediction > 0.5))
    return final_prediction


lls = []
wghts = []
for i in range(1000):
    bounds = [(0, 1) for _ in range(len(predictions))]
    starting_values = np.ones(len_prediction) / len_prediction
    res = minimize(metric_func, starting_values, method='L-BFGS-B', bounds=bounds)
    
    lls.append(res['fun'])
    wghts.append(res['x'])

In [75]:
bestSC = np.min(lls)
bestWght = wghts[np.argmin(lls)]

print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))


 Ensemble Score: 0.8361391694725028

 Best Weights: [0.25 0.25 0.25 0.25]


## Optimize Threshold

In [72]:
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

prediction = oof.mean(axis=1).to_numpy()

def metric_func(weight):
    final_prediction = accuracy_score(y, (prediction > weight))
    return final_prediction

lls = []
wghts = []
for i in range(1000):
    starting_values = np.array([0.5])
    res = minimize(metric_func, starting_values, method='L-BFGS-B', bounds=bounds)
    
    lls.append(res['fun'])
    wghts.append(res['x'])

In [73]:
bestSC = np.min(lls)
bestWght = wghts[np.argmin(lls)]

print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))


 Ensemble Score: 0.8361391694725028

 Best Weights: [0.5]
